# Investor ranking based on their ‘seed’ investment:  

This notebook focuses on companies that that had both the ‘seed’ and ‘venture a’ funding rounds. And by analyzing the percentage change in the money raised on their ‘seed’ and ‘venture a’ round, an “operating score is generated” for 27903 investors. 

This concept can be expanded for (‘venture a’ to ‘venture b’), (‘venture b’ to ‘venture c’) and (‘venture c’ to ‘venture d’) for the period of the last 5, 10 and 20 years. 



In [1]:
## Importing the required liberties 
from utils import *
%matplotlib inline

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning:

`scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.



plotly  2.4.0


 ## Reading the data:

Two data source is used, one is funding and the other one is ‘status’ of the company. I only looked at the companies that are still operating and raised more than 10,000 USD.  



In [2]:
df_main = pd.read_csv('companies_funding_dollar_adjusted_status.csv')

df_main_operating = df_main[df_main.status == 'operating']
df_main_operating = df_main_operating[df_main_operating.amount_raised > 10000]

##  Making a new dataset with data from companies with 'seed' and 'venture a'  round type:

In [3]:
df_main_operating_seed = df_main_operating[(df_main_operating.round_type_fixed == 'seed') ]
df_main_operating_ven = df_main_operating[(df_main_operating.round_type_fixed == 'venture a')]
df_main_operating_ve_seed =  pd.concat([df_main_operating_seed, df_main_operating_ven])

In [4]:
len(df_main_operating_ve_seed.ds_uuid.value_counts())

27903

In [5]:
display(DataFrameSummary(df_main_operating_ve_seed).summary())

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/pandas_summary/__init__.py:111: DeprecationWarning:

pandas.core.common.is_numeric_dtype is deprecated. import from the public API: pandas.api.types.is_numeric_dtype instead

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/pandas_summary/__init__.py:111: DeprecationWarning:

pandas.core.common.is_numeric_dtype is deprecated. import from the public API: pandas.api.types.is_numeric_dtype instead

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/pandas_summary/__init__.py:111: DeprecationWarning:

pandas.core.common.is_numeric_dtype is deprecated. import from the public API: pandas.api.types.is_numeric_dtype instead

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/pandas_summary/__init__.py:111: DeprecationWarning:

pandas.core.common.is_numeric_dtype is deprecated. import from the public API: pandas.api.types.is_numeric_dtype instead

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/pandas_summary/__init__.p

,Unnamed: 0,Unnamed: 0.1,ds_uuid,investor,date,amount_raised,round_type,update_status,source,round_id,...,date_pars,year,month,week,cpi1,adjusted_amount_raised,month_s,month_n,round_type_fixed,status
count,80539,80539,NaN,NaN,NaN,80539,NaN,NaN,NaN,80539,...,NaN,80539,80539,80539,80363,80363,80539,NaN,NaN,NaN
mean,123167,222555,NaN,NaN,NaN,4.45113e+06,NaN,NaN,NaN,2.19293,...,NaN,2013.84,6.48117,26.7777,1.02975,4.61567e+06,6.48117,NaN,NaN,NaN
std,71404.6,121222,NaN,NaN,NaN,1.66011e+07,NaN,NaN,NaN,1.53767,...,NaN,3.04267,3.43523,15.0219,0.063848,1.73742e+07,3.43523,NaN,NaN,NaN
min,1,7,NaN,NaN,NaN,10200,NaN,NaN,NaN,1,...,NaN,1982,1,1,0.979141,10500,1,NaN,NaN,NaN
25%,61651.5,123122,NaN,NaN,NaN,600000,NaN,NaN,NaN,1,...,NaN,2013,4,14,1,623128,4,NaN,NaN,NaN
50%,122479,233980,NaN,NaN,NaN,1.85e+06,NaN,NaN,NaN,2,...,NaN,2015,7,27,1.01262,1.9e+06,7,NaN,NaN,NaN
75%,185156,325322,NaN,NaN,NaN,5e+06,NaN,NaN,NaN,3,...,NaN,2016,9,40,1.03026,5e+06,9,NaN,NaN,NaN
max,247222,423917,NaN,NaN,NaN,1.08e+09,NaN,NaN,NaN,10,...,NaN,2018,12,53,2.48712,1.08e+09,12,NaN,NaN,NaN
counts,80539,80539,80539,80538,80539,80539,80539,80440,80539,80539,...,80539,80539,80539,80539,80363,80363,80539,80539,80539,80539
uniques,80539,80539,27903,19098,4182,2946,4,1,1,10,...,4182,32,12,53,31,6474,12,12,2,1


## Calculating percentage change betweem 'seed' and 'venture a':

In the following section, I am calculating the percentage change between the money raised between ‘seed’ and ‘venture a’ round.  



In [6]:
money = df_main_operating_ve_seed.groupby([ 'clean_url','date','round_type_fixed','investor'],as_index = False)[['adjusted_amount_raised']].mean()
money2 = money.groupby([ 'clean_url','round_type_fixed','date'],as_index = False)[['adjusted_amount_raised']].mean()
money22 = money2.groupby([ 'clean_url','round_type_fixed'],as_index = False)[['adjusted_amount_raised']].sum()

money22['difference_money_raised'] = money22.groupby([ 'clean_url',],as_index = False)[['adjusted_amount_raised']].diff()
money22['pct_money_raised'] = money22.groupby([ 'clean_url'],as_index = False)[['adjusted_amount_raised']].pct_change(1)

In [7]:
money3 = money22.groupby([ 'clean_url'],as_index = True)[['pct_money_raised']].sum()
money3_as = money3.sort_values('pct_money_raised', ascending=False)

iplot(money3_as[:100].iplot(asFigure=True,
                 kind='bar',
                 xTitle='',
                 yTitle='',
                 title=''))

In [8]:
iplot(money3_as.iplot(asFigure=True,
                 kind='histogram',
                 xTitle='',
                 yTitle='',
                 title=''))

In [9]:
clean = money3_as.dropna()
clean.describe()

,pct_money_raised
count,27903.000000
mean,1.142641
std,13.705492
min,-1.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,790.397849


## Adding the company score back to the original dataset:

In this section, I will rank the investors based on the performance of their investment in seed companies. 



In [10]:
money4 = money3_as.reset_index()
money4.columns

Index(['clean_url', 'pct_money_raised'], dtype='object')

In [11]:
df_main_operating_seed = join_df(df_main_operating_seed, money4, "clean_url", "clean_url", col_on = ['clean_url','pct_money_raised'])

In [12]:
df_main_operating_seed.pct_money_raised.describe()

count    49545.000000
mean         1.409831
std         12.778079
min         -1.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        790.397849
Name: pct_money_raised, dtype: float64

In [13]:
df_main_operating_seed_nan = df_main_operating_seed.dropna()

In [14]:
df_main_operating_seed_nan.shape, df_main_operating_seed.shape

((49382, 24), (49545, 24))

## Looking at the best performing investors: 

In [15]:
df_main_operating_seed_nan['year_s'] = df_main_operating_seed_nan.year.apply(lambda x: str(x))

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



## Looking at the best performing of all time: 

In [16]:
df_g1 = df_main_operating_seed_nan.groupby([ 'investor'],)[['pct_money_raised']].mean()
df_des = df_g1.sort_values('pct_money_raised', ascending=False)
iplot(df_des[:200].iplot(asFigure=True,
                 kind='bar',
                 xTitle='',
                 yTitle='',
                 title='Top performing investors based on seed to venture A performance '))

In [17]:
df_des = df_g1.sort_values('pct_money_raised', ascending=True)
iplot(df_des[:200].iplot(asFigure=True,
                 kind='bar',
                 xTitle='',
                 yTitle='',
                 title='Worst performing investors based on seed to venture A performance'))

In [18]:
df_g1.loc['Index Ventures']

pct_money_raised    1.077677
Name: Index Ventures, dtype: float64